<a href="https://www.kaggle.com/seanpharris/cryptoanalysispipe?scriptVersionId=88753878" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Crypto Currency Exploration
# Data from Alpha Vantage - RapidAPI
## Documentation
* https://github.com/seanpharris/CryptoAnalysisPipe
## Contact info
* Sean Pharris - pharris.sean@gmail.com

### Process
* This process has a scheduled run every Wednesday at midnight
* The loop below calls the API and gathers the last week's currency records 
* It appends the last weeks data to the datafame within the dictionary and then to the CSV
* Each currency has its own CSV file which can be easily added to a notebook by going to "Output files" from this notebook

Side note - Alpha Vantage keeps records for the most recent 1000 days; I chose to append the current data instead of capturing the data passed through the API so that each currency's data can exceed 1000 days.

In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings('ignore')

/kaggle/input/crypto-files/Crypto-proj/Ontology.csv
/kaggle/input/crypto-files/Crypto-proj/Binance-USD.csv
/kaggle/input/crypto-files/Crypto-proj/Monero.csv
/kaggle/input/crypto-files/Crypto-proj/Storm.csv
/kaggle/input/crypto-files/Crypto-proj/Enjin-Coin.csv
/kaggle/input/crypto-files/Crypto-proj/Qtum.csv
/kaggle/input/crypto-files/Crypto-proj/Civic.csv
/kaggle/input/crypto-files/Crypto-proj/Compound.csv
/kaggle/input/crypto-files/Crypto-proj/DATAcoin.csv
/kaggle/input/crypto-files/Crypto-proj/Zcash.csv
/kaggle/input/crypto-files/Crypto-proj/ICON.csv
/kaggle/input/crypto-files/Crypto-proj/Tezos.csv
/kaggle/input/crypto-files/Crypto-proj/Komodo.csv
/kaggle/input/crypto-files/Crypto-proj/Storj.csv
/kaggle/input/crypto-files/Crypto-proj/aelf.csv
/kaggle/input/crypto-files/Crypto-proj/TerraUSD.csv
/kaggle/input/crypto-files/Crypto-proj/BitTorrent.csv
/kaggle/input/crypto-files/Crypto-proj/Waves.csv
/kaggle/input/crypto-files/Crypto-proj/ZenCash.csv
/kaggle/input/crypto-files/Crypto-proj/C

In [2]:
pip install alpha_vantage

Note: you may need to restart the kernel to use updated packages.


In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
API_KEY = user_secrets.get_secret("API_KEY")

In [4]:
# import digital currency list (Contains all Crypto names and codes)
df = pd.read_csv('/kaggle/input/curr-list/currency_list.csv')

In [5]:
# df_dict is a dictionary that contatins all currency dataframes
df_dict = {'name':[]}

for name in df['currency name']:
    df_dict['name'].append(name)
    
for name in df['currency name']:
    csv_path = "/kaggle/input/crypto-files/Crypto-proj/" + name + ".csv"
    df_dict[name] = pd.read_csv(csv_path)
    df_dict[name].sort_values(by='date', inplace=True)

### Example Dataframe before update

In [6]:
df_dict['Aave']

,date,1a. open (CNY),1b. open (USD),2a. high (CNY),2b. high (USD),3a. low (CNY),3b. low (USD),4a. close (CNY),4b. close (USD),5. volume,6. market cap (USD)
482,2020-10-15,327.264519,51.430,375.434700,59.000,272.011985,42.747,273.615537,42.999,98648.870,98648.870
481,2020-10-16,273.730076,43.017,278.712540,43.800,248.754124,39.092,251.102181,39.461,90836.412,90836.412
480,2020-10-17,250.529484,39.371,275.117276,43.235,243.116240,38.206,263.217904,41.365,41971.143,41971.143
479,2020-10-18,263.179725,41.359,263.599702,41.425,250.892192,39.428,259.304475,40.750,30917.953,30917.953
478,2020-10-19,259.749906,40.820,260.895300,41.000,228.518830,35.912,228.951534,35.980,86475.936,86475.936
...,...,...,...,...,...,...,...,...,...,...,...
4,2022-02-05,1061.398440,166.800,1104.032550,173.500,1044.217530,164.100,1057.580460,166.200,100784.207,100784.207
3,2022-02-06,1057.580460,166.200,1110.395850,174.500,1051.853490,165.300,1104.032550,173.500,91113.374,91113.374
2,2022-02-07,1104.668880,173.600,1172.119860,184.200,1080.488340,169.800,1158.756930,182.100,119127.010,119127.010
1,2022-02-08,1160.029590,182.300,1226.207910,192.700,1104.032550,173.500,1170.210870,183.900,154091.746,154091.746


In [7]:
from datetime import date, timedelta


yesterday = (date.today() - timedelta(days=1)).strftime("%Y-%m-%d")
yesterday

'2022-02-24'

In [8]:
from datetime import date

starting = df_dict['Aave']['date'].max()
datelist = pd.date_range(start=starting, end=yesterday)
date = datelist.strftime("%Y-%m-%d")
date = date.drop(date[0])
for day in date:
    print(day)

2022-02-10
2022-02-11
2022-02-12
2022-02-13
2022-02-14
2022-02-15
2022-02-16
2022-02-17
2022-02-18
2022-02-19
2022-02-20
2022-02-21
2022-02-22
2022-02-23
2022-02-24


In [28]:
import requests
import time
import json
from csv import DictWriter
from alpha_vantage.cryptocurrencies import CryptoCurrencies
from datetime import date

today = (date.today()).strftime("%Y-%m-%d")

headersCSV = ['date','1a. open (CNY)', '1b. open (USD)', '2a. high (CNY)', '2b. high (USD)',
       '3a. low (CNY)', '3b. low (USD)', '4a. close (CNY)', '4b. close (USD)',
       '5. volume', '6. market cap (USD)']

for n, code in enumerate(df['currency code']):
       
    time.sleep(15) # 15 second delay is due to the ability to only make 5 API calls per minute
    
    data = None
    working_currency = df['currency name'][n]
    currency_csv = working_currency + ".csv"
    csv_path = "./" + currency_csv
    
    starting_date = df_dict[working_currency]['date'].max()
    
    datelist = pd.date_range(start=starting_date, end=yesterday)
    datelist = datelist.strftime("%Y-%m-%d")
    datelist = datelist.drop(datelist[0])
    
    df_dict[working_currency].to_csv(currency_csv, index=False)
    
    try:

        cc = CryptoCurrencies(key= API_KEY, rapidapi=True, output_format='pandas')

        data, meta_data = cc.get_digital_currency_daily(symbol=code, market='CNY')

        for day in datelist:
            new_data = data.loc[day].to_numpy()
            open_cny = new_data[0][0]
            open_usd = new_data[0][1]
            high_cny = new_data[0][2]
            high_usd = new_data[0][3]
            low_cny = new_data[0][4]
            low_usd = new_data[0][5]
            close_cny = new_data[0][6]
            close_usd = new_data[0][7]
            volume = new_data[0][8]
            market_cap = new_data[0][9]

            dict={'date':day, '1a. open (CNY)': open_cny, '1b. open (USD)': open_usd, '2a. high (CNY)': high_cny, '2b. high (USD)': high_usd,
               '3a. low (CNY)': low_cny, '3b. low (USD)': low_usd, '4a. close (CNY)': close_cny, '4b. close (USD)': close_usd,
               '5. volume': volume, '6. market cap (USD)': market_cap}

            with open(csv_path, 'a', newline='') as f_object:  

                dictwriter_object = DictWriter(f_object, fieldnames=headersCSV)
                dictwriter_object.writerow(dict)

        f_object.close()

        df_dict[working_currency] = pd.read_csv(csv_path)
        df_dict[working_currency].sort_values(by='date', inplace=True)

        print('Crypto:', n, '- SUCCESS for ', working_currency)
    
        
    except(IndexError):
        datelist.drop(datelist.max())
        continue
        
    except(ValueError):
        print('Crypto:', n, '- Could not retrieve data for ', working_currency)
        pass


Crypto:  0 - SUCCESS for  Aave
Crypto:  1 - SUCCESS for  Cardano
Crypto:  2 - SUCCESS for  Aion
Crypto:  3 - SUCCESS for  Algorand
Crypto:  4 - SUCCESS for  Synereo
Crypto:  5 - SUCCESS for  Anoncoin
Crypto:  6 - SUCCESS for  Aragon
Crypto:  7 - SUCCESS for  Ardor
Crypto:  8 - SUCCESS for  ATMChain
Crypto:  9 - SUCCESS for  Cosmos
Crypto:  10 - SUCCESS for  Avalanche
Crypto:  11 - SUCCESS for  Band Protocol
Crypto:  12 - SUCCESS for  Basic-Attention-Token
Crypto:  14 - SUCCESS for  Bitcoin-Cash
Crypto:  15 - SUCCESS for  Bluzelle
Crypto:  16 - SUCCESS for  Binance-Coin
Crypto:  17 - SUCCESS for  Bancor-Network-Token
Crypto:  18 - SUCCESS for  Bitcoin
Crypto:  19 - SUCCESS for  BitShares
Crypto:  21 - SUCCESS for  Binance-USD
Crypto:  22 - SUCCESS for  PancakeSwap
Crypto:  23 - SUCCESS for  Compound
Crypto:  24 - SUCCESS for  Cortex
Crypto:  25 - SUCCESS for  Civic
Crypto:  27 - SUCCESS for  Darcrus
Crypto:  28 - SUCCESS for  Dash
Crypto:  29 - SUCCESS for  DATAcoin
Crypto:  30 - SUCCES

### Example DataFrame after update

In [10]:
df_dict['Aave']

,date,1a. open (CNY),1b. open (USD),2a. high (CNY),2b. high (USD),3a. low (CNY),3b. low (USD),4a. close (CNY),4b. close (USD),5. volume,6. market cap (USD)
0,2020-10-15,327.264519,51.430,375.434700,59.000,272.011985,42.747,273.615537,42.999,98648.8700,98648.8700
1,2020-10-16,273.730076,43.017,278.712540,43.800,248.754124,39.092,251.102181,39.461,90836.4120,90836.4120
2,2020-10-17,250.529484,39.371,275.117276,43.235,243.116240,38.206,263.217904,41.365,41971.1430,41971.1430
3,2020-10-18,263.179725,41.359,263.599702,41.425,250.892192,39.428,259.304475,40.750,30917.9530,30917.9530
4,2020-10-19,259.749906,40.820,260.895300,41.000,228.518830,35.912,228.951534,35.980,86475.9360,86475.9360
...,...,...,...,...,...,...,...,...,...,...,...
493,2022-02-20,923.589260,146.200,924.220990,146.300,853.467230,135.100,869.260480,137.600,57594.7330,57594.7330
494,2022-02-21,868.628750,137.500,914.113310,144.700,807.350940,127.800,812.404780,128.600,128162.9080,128162.9080
495,2022-02-22,812.404780,128.600,865.470100,137.000,794.716340,125.800,850.940310,134.700,94375.6130,94375.6130
496,2022-02-23,850.940310,134.700,904.005630,143.100,821.880730,130.100,825.039380,130.600,94147.9000,94147.9000
